In [ ]:
parse_urls = [
    {   'url': "https://www.fdic.gov/news/press-releases/2021/",
        'source': 'FDIC',
        'tags': 'general',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
    {   'url': "https://www.fdic.gov/news/press-releases/2022/",
        'source': 'FDIC',
        'tags': 'general',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
        ]


response = requests.get(parse_urls[0]['url'])
# print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')
soup.find_all('ul', class_='press_releases')
# articles = soup.find_all('tr', class_='pr-list-page-row')


In [ ]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser
import pandas as pd

parse_urls = [
    {   'url': "https://www.sec.gov/news/pressreleases?aId=&combine=crypto&year=2021&month=All",
        'source': 'SEC',
        'tags': 'crypto',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
    {   'url': "https://www.sec.gov/news/pressreleases?aId=&combine=&year=2021&month=All",
        'source': 'SEC',
        'tags': 'general',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
    {   'url': "https://www.sec.gov/news/pressreleases?aId=&combine=crypto&year=2022&month=All",
        'source': 'SEC',
        'tags': 'crypto',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
    {   'url': "https://www.sec.gov/news/pressreleases?aId=&combine=&year=2022&month=All",
        'source': 'SEC',
        'tags': 'general',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
    {   'url': "https://www.fdic.gov/news/press-releases/2021/",
        'source': 'FDIC',
        'tags': 'general',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
    {   'url': "https://www.fdic.gov/news/press-releases/2022/",
        'source': 'FDIC',
        'tags': 'general',
        'type': 'press release',
        'language': 'en',
        'country': 'US'
    },
        ]

url_parsed_dict = dict()
regulatory_news_df = pd.DataFrame()
debugging = True

for parse_url in parse_urls:

    if parse_url['source'] == 'SEC':

        continue

        response = requests.get(parse_url['url'])
        # print(response.status_code)
        soup = BeautifulSoup(response.content, 'html.parser')

        articles = soup.find_all('tr', class_='pr-list-page-row')

        for article in reversed(articles):


            processed_row = dict() 
            row = article.find_all('td')

            # 1. Time
            processed_row['date'] = parser.parse(row[1].find('time')['datetime']).strftime('%Y-%m-%d %H:%M:%S')
            # 2. URL
            processed_row['url'] = "https://www.sec.gov" + row[2].find('a')['href']
            if processed_row['url'] not in url_parsed_dict:
                url_parsed_dict[processed_row['url']] = True
            else:
                continue
            # 3. Title
            processed_row['title'] = row[2].find('a').text.strip()
            # 4. Text
            article_soup = BeautifulSoup(requests.get(processed_row['url']).content, 'html.parser')
            processed_row['text'] = article_soup.find('div', class_='article-body').text.strip()
            # 5. Source
            processed_row['source'] = parse_url['source']
            # 6. Tags
            processed_row['tags'] = parse_url['tags']
            # 7. Type
            processed_row['type'] = parse_url['type']
            # 8. Language
            processed_row['language'] = parse_url['language']
            # 9. Country
            processed_row['country'] = parse_url['country']

            if debugging:
                print(processed_row)

            processed_row_df = pd.DataFrame(processed_row, index=[0])
            regulatory_news_df = pd.concat([regulatory_news_df, processed_row_df], ignore_index=True, axis=0)

regulatory_news_df


In [ ]:
regulatory_news_df

In [ ]:
regulatory_news_df.to_csv('regulatory_news.csv', index=False)